# Build a Hybrid ReAct + Planner AI Agent with Flyte

Welcome to the **ultimate multi-agent pattern** - combining the best of both worlds!

## 🎯 What We'll Build

A **hybrid agent system** that:
- **Adapts** like ReAct (changes strategy based on results)
- **Parallelizes** like Planner (executes independent tasks simultaneously)
- **Reflects** after each iteration to improve decision-making
- **Creates mini-plans** that balance efficiency and flexibility

Think of it as: **"Smart iterations with parallel execution inside each iteration"** 🚀

## 📚 Learning Objectives

By the end of this tutorial, you'll understand:

1. **Hybrid Pattern** - Combining ReAct's adaptability with Planner's parallelism
2. **Mini-Planning** - Creating small, executable plans per iteration
3. **Iterative Refinement** - Adapting strategy while leveraging concurrency
4. **Dependency Management** - Enabling parallelism within adaptive workflows
5. **Production Optimization** - Best practices for real-world agent systems

---

## 🏗️ Evolution of Agent Patterns

Let's see how we got here:

### 1. Static Planner:
```
User Request → Create FULL plan → Execute with parallelism
```
✅ **Pros:** Maximum parallelism, very efficient  
❌ **Cons:** Can't adapt, fails if assumptions are wrong

### 2. ReAct Agent:
```
Think → Act (ONE step) → Observe → Reflect → Repeat
```
✅ **Pros:** Fully adaptive, handles uncertainty  
❌ **Cons:** Purely sequential, slower for independent tasks

### 3. Hybrid (This Tutorial!):
```
Think → Mini-Plan (1-5 steps) → Execute in PARALLEL → Reflect → Repeat
```
✅ **Pros:** Adaptive AND efficient, best of both worlds  
✅ **Use Case:** Production-ready for complex, uncertain tasks

---

## 🔄 The Hybrid Loop

```
┌──────────────────────────────────────┐
│  GOAL: Calculate X, Y, Z, then sum   │
└───────────────┬──────────────────────┘
                ↓
╔═══════════════════════════════════════╗
║         ITERATION 1                   ║
╠═══════════════════════════════════════╣
║  💭 THOUGHT: "I need X, Y, Z first"  ║
║                                       ║
║  📋 MINI-PLAN:                        ║
║    Step 0: Calculate X (deps: [])    ║ ┐
║    Step 1: Calculate Y (deps: [])    ║ ├─ PARALLEL!
║    Step 2: Calculate Z (deps: [])    ║ ┘
║                                       ║
║  🚀 EXECUTE: All 3 in parallel       ║
║     → X=10, Y=20, Z=30                ║
║                                       ║
║  🤔 REFLECTION: "Got all values!     ║
║     Now need to sum them"             ║
╚═══════════════════════════════════════╝
                ↓
╔═══════════════════════════════════════╗
║         ITERATION 2                   ║
╠═══════════════════════════════════════╣
║  💭 THOUGHT: "Sum X, Y, Z"           ║
║                                       ║
║  📋 MINI-PLAN:                        ║
║    Step 0: Sum 10+20+30 (deps: [])   ║
║                                       ║
║  🚀 EXECUTE: Calculate sum            ║
║     → Result=60                       ║
║                                       ║
║  🤔 REFLECTION: "Goal achieved!"     ║
╚═══════════════════════════════════════╝
                ↓
         ✅ DONE: 60
```

**Key Insight:** Each iteration can execute multiple steps in parallel, but we can still adapt between iterations!

## ⚙️ Setup and Prerequisites

This tutorial builds on both previous patterns:
- Same agents as before (math, string, web_search, code, weather)
- Same dependency-aware execution engine
- Same Flyte infrastructure

**What's NEW:** The iteration pattern that creates mini-plans!

In [ ]:
# Utility to display code files with syntax highlighting
def print_code_file(file_path):
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import HtmlFormatter
    from IPython.display import HTML

    code = open(file_path).read()
    formatter = HtmlFormatter(style="one-dark", noclasses=True)
    html = highlight(code, PythonLexer(), formatter)
    display(HTML(html))

---

## 🧠 The Hybrid Pattern - Core Concepts

Let's examine what makes this pattern special:

### 1️⃣ **Data Models**

We track **iterations** (not individual steps):

```python
@dataclass
class HybridIteration:
    """Single iteration of the hybrid workflow"""
    iteration_number: int
    thought: str                      # "I need X, Y, Z first"
    plan_steps: List[AgentStep]       # Mini-plan (1-5 steps)
    step_results: List[Dict]          # Results from each step
    reflection: str                   # "Got all values!"
    goal_achieved: bool = False

@dataclass
class HybridResult:
    """Final result from hybrid workflow"""
    goal: str
    iterations: List[HybridIteration]  # Full execution trace
    final_answer: str
    total_iterations: int
    total_steps_executed: int          # Across all iterations
    goal_achieved: bool
```

**Notice:** Each iteration can contain **multiple steps** that execute in parallel!

### 2️⃣ **The Mini-Planning Prompt**

This is the secret sauce - we ask the LLM to create a **small plan** for this iteration:

```python
system_msg = f"""
You are a hybrid ReAct + Planner agent.

Goal: {user_goal}

Previous iterations:
{history_text}

Your task: Think about what to do next, then create a MINI-PLAN.

Respond in JSON format:
{{
  "thought": "Your reasoning about the current state and what's needed",
  "goal_achieved": false,
  "plan_steps": [
    {{"agent": "agent_name", "task": "specific task", "dependencies": []}},
    {{"agent": "agent_name", "task": "another task", "dependencies": []}}
  ],
  "final_answer": null
}}

MINI-PLAN RULES:
- Include 1-5 steps maximum per iteration
- Steps with empty dependencies [] run in PARALLEL
- Use dependencies: [0] to make step 1 wait for step 0
- Be strategic: group independent tasks to leverage parallelism
- You can do more steps in the NEXT iteration if needed
"""
```

**Key Design Choices:**
- ✅ Mini-plans (1-5 steps) keep iterations focused
- ✅ Dependencies enable parallelism within iterations
- ✅ Can adapt between iterations based on results
- ✅ LLM learns to balance breadth vs. depth

### 3️⃣ **Executing Mini-Plans with Parallelism**

The execution engine is the same as the static planner:

```python
async def execute_mini_plan(plan_steps: List[AgentStep]) -> List[Dict]:
    """
    Execute a mini-plan with dependency-aware parallel execution.
    """
    completed_results: Dict[int, Dict] = {}
    pending_steps = list(enumerate(plan_steps))
    
    while pending_steps:
        # Find steps that can execute now (dependencies satisfied)
        ready_steps = [
            (idx, step) for idx, step in pending_steps
            if all(dep_idx in completed_results for dep_idx in step.dependencies)
        ]
        
        # Execute ready steps in parallel using asyncio.gather
        results = await asyncio.gather(*[
            execute_step(idx, step) for idx, step in ready_steps
        ])
        
        # Store completed results
        for step_idx, result in results:
            completed_results[step_idx] = result
        
        # Update pending steps
        pending_steps = [s for s in pending_steps if s not in ready_steps]
    
    return [completed_results[i] for i in range(len(plan_steps))]
```

**What's happening:**
1. Find all steps with satisfied dependencies
2. Execute them ALL in parallel
3. Record results
4. Repeat until all steps complete

**This is the SAME engine as the static planner, but now it runs per-iteration!**

### 4️⃣ **The Iteration Loop**

Each iteration follows this pattern:

```python
for iter_num in range(1, max_iterations + 1):
    # 1. Build context from previous iterations
    history_text = format_previous_iterations(iterations[-2:])
    
    # 2. Ask LLM: "What's the mini-plan for this iteration?"
    decision = await llm.create_mini_plan(
        goal=user_goal,
        history=history_text
    )
    
    # 3. Check if done
    if decision["goal_achieved"]:
        break
    
    # 4. Execute the mini-plan with parallel execution
    step_results = await execute_mini_plan(decision["plan_steps"])
    
    # 5. Reflect on ALL results from this iteration
    reflection = await llm.reflect(
        thought=decision["thought"],
        steps=decision["plan_steps"],
        results=step_results
    )
    
    # 6. Record this iteration
    iterations.append(HybridIteration(...))
```

**Notice:** We execute MULTIPLE steps per iteration, but we can still adapt the strategy between iterations!

### 5️⃣ **Reflection - Learning from Multiple Results**

After each iteration, we reflect on ALL results:

```python
reflection_prompt = f"""
Reflect on the results of this iteration:

Goal: {user_goal}
Thought: {thought}
Steps executed: {len(step_results)}

Results:
{results_summary}

Provide a brief reflection (2-3 sentences):
1. What did we learn from these results?
2. Are we closer to the goal?
3. What should we do next (or are we done)?
"""
```

**Why reflect on multiple results?**
- Synthesize insights from parallel executions
- Identify patterns or problems
- Plan next iteration more effectively
- Decide if goal is achieved

---

## 📝 The Complete Hybrid Workflow

Let's look at the full implementation:

### Key Features:

1. **Adaptive Iterations** - Strategy changes between iterations
2. **Parallel Execution** - Independent tasks run simultaneously within iterations
3. **Mini-Planning** - 1-5 steps per iteration (focused but flexible)
4. **Context Accumulation** - Previous iterations inform future decisions
5. **Dependency Management** - Full DAG execution per iteration
6. **Smart Reflection** - Synthesizes multiple results

### Code Walkthrough:

In [ ]:
print_code_file("workflows/flyte_react_planner.py")

---

## 🔍 Code Deep Dive: Critical Sections

Let's examine the most important parts:

### 🧩 Building Context from Previous Iterations

```python
if context_history:
    history_text = "\n\n".join([
        f"Iteration {h['iteration']}:\n"
        f"Thought: {h['thought']}\n"
        f"Executed {len(h['plan_steps'])} step(s)\n"
        f"Results: {h['summary']}\n"
        f"Reflection: {h['reflection']}"
        for h in context_history[-2:]  # Last 2 iterations
    ])
```

**Notice:** We track ITERATIONS (not steps), and include summary of all steps in each iteration.

---

### 🎯 Parsing Mini-Plans

```python
plan_steps = [
    AgentStep(
        agent=step["agent"],
        task=step["task"],
        dependencies=step.get("dependencies", [])
    )
    for step in decision["plan_steps"]
]
```

**Key:** Each step can have dependencies, enabling parallelism within the mini-plan!

---

### 🚀 Dependency-Aware Execution

```python
async def execute_step(step_idx: int, step: AgentStep) -> tuple:
    # If this step has dependencies, augment task with results
    task = step.task
    if step.dependencies:
        dep_results = []
        for dep_idx in step.dependencies:
            dep_result = completed_results[dep_idx]
            dep_results.append(f"Result from step {dep_idx}: {dep_result['observation']}")
        task = f"Context:\n" + "\n".join(dep_results) + f"\n\nYour task: {task}"
    
    # Route to appropriate agent
    if step.agent == "math":
        result = await math_agent(task)
        observation = result.final_result
    # ... other agents ...
    
    return step_idx, {
        "agent": step.agent,
        "task": step.task,
        "observation": observation
    }
```

**This is the SAME pattern as the static planner** - we're reusing the dependency engine!

---

## 🚀 Running the Hybrid Workflow

### Local Execution (Development):
```bash
python -m workflows.flyte_react_planner --local --request "Your goal here" --max-iterations 10
```

### Remote Execution (Production):
```bash
python -m workflows.flyte_react_planner --request "Your goal here" --max-iterations 10
```

### 📝 Example Prompts to Try:

#### Simple Parallel Task:
```python
"Calculate 5 factorial and count words in 'hello world', then add the results"
```
**Expected behavior:**
- Iteration 1: [factorial, word_count] in parallel → 120, 2
- Iteration 2: [add 120 + 2] → 122

---

#### Complex Multi-Agent:
```python
"Calculate 10 factorial, count words in 'hello world', search for 'Python async', then write code to sum the numeric results"
```
**Expected behavior:**
- Iteration 1: [math, string, web_search] all in parallel
- Iteration 2: [code agent to sum results]

---

#### Adaptive Research:
```python
"Find the GDP of France and Germany, then compare them with a calculation"
```
**Expected behavior:**
- Iteration 1: [search France GDP, search Germany GDP] in parallel
- Iteration 2: [code to compare and calculate difference]

**Notice:** The agent can adapt if search results are unclear!

Let's run an example:

In [ ]:
# Run a hybrid workflow example
!python -m workflows.flyte_react_planner --local --request "Calculate 10 factorial, count words in 'hello world', search for 'Python async', then write code to sum the numeric results" --max-iterations 5

---

## 📊 Hybrid in Action: Step-by-Step Example

Let's trace through a real execution:

**Goal:** "Calculate 10 factorial, count words in 'hello world', search for 'Python async', then sum numeric results"

---

### Iteration 1:

**💭 Thought:**
```
"I need to get the three values first: factorial, word count, and search results"
```

**📋 Mini-Plan:**
```json
[
  {"agent": "math", "task": "Calculate 10 factorial", "dependencies": []},
  {"agent": "string", "task": "Count words in 'hello world'", "dependencies": []},
  {"agent": "web_search", "task": "Search for Python async", "dependencies": []}
]
```

**🚀 Execution:** All 3 steps run **IN PARALLEL**

**📊 Results:**
- Step 0 (math): `3628800`
- Step 1 (string): `2`
- Step 2 (web_search): `Found documentation about asyncio...`

**🤔 Reflection:**
```
"Successfully obtained factorial (3628800) and word count (2). 
The search results are informational. Now I need to sum the numeric results."
```

---

### Iteration 2:

**💭 Thought:**
```
"I have the numeric results (3628800 and 2). I should use the code agent to sum them."
```

**📋 Mini-Plan:**
```json
[
  {
    "agent": "code",
    "task": "Sum 3628800 and 2",
    "dependencies": []
  }
]
```

**🚀 Execution:** Single step

**📊 Results:**
- Step 0 (code): `3628802`

**🤔 Reflection:**
```
"Successfully calculated the sum. Goal achieved!"
```

---

### Iteration 3:

**💭 Thought:**
```
"I have completed all tasks. The sum of numeric results is 3628802."
```

**✅ Goal Achieved:**
```
"10 factorial = 3,628,800
Word count = 2
Sum of numeric results = 3,628,802"
```

---

### Summary:
- **Total Iterations:** 3
- **Total Steps:** 4 (3 in parallel in iteration 1, 1 in iteration 2)
- **Parallelism:** 3 agents ran simultaneously in iteration 1
- **Adaptability:** Strategy adjusted after seeing iteration 1 results

**Notice how the agent:**
1. ✅ Identified independent tasks and ran them in parallel
2. ✅ Adapted the plan based on results from iteration 1
3. ✅ Used appropriate agents for each subtask
4. ✅ Synthesized multiple results into final answer

---

## 🎓 Key Takeaways

### What Makes Hybrid Special:

1. **⚡ Best Performance**
   - Parallelizes independent tasks within iterations
   - Faster than pure ReAct (no sequential bottleneck)
   - Nearly as fast as static planner for known tasks

2. **🔄 Full Adaptability**
   - Can change strategy between iterations
   - Handles unexpected results gracefully
   - More flexible than static planner

3. **🧠 Smart Planning**
   - LLM learns to group independent tasks
   - Creates mini-plans optimized for parallelism
   - Balances breadth (parallelism) vs. depth (dependencies)

4. **📊 Rich Context**
   - Reflects on multiple results per iteration
   - Synthesizes insights from parallel executions
   - Builds understanding progressively

5. **🎯 Production Ready**
   - Efficient resource utilization
   - Handles complex, uncertain tasks
   - Full observability and traceability

---

### Pattern Comparison:

| Feature | Static Planner | ReAct | **Hybrid** |
|---------|----------------|-------|------------|
| Parallelism | ✅✅✅ Full | ❌ None | ✅✅ Per-iteration |
| Adaptability | ❌ None | ✅✅✅ Full | ✅✅ Between iterations |
| Speed | ⚡⚡⚡ Fastest | 🐌 Slowest | ⚡⚡ Fast |
| Flexibility | 🔒 Rigid | 🦎 Fully adaptive | 🦎 Adaptive + efficient |
| Best For | Known workflows | Exploration | **Complex production tasks** |
| Uncertainty Handling | ❌ Poor | ✅✅✅ Excellent | ✅✅ Very good |

---

### When to Use Each Pattern:

| Scenario | Best Approach | Why |
|----------|---------------|-----|
| Predictable workflow | Static Planner | Maximum parallelism |
| Highly uncertain | ReAct | Full exploration |
| **Production (complex)** | **Hybrid** | **Balance of speed + adaptability** |
| Research + calculation | **Hybrid** | **Parallel research, adaptive synthesis** |
| Interactive debugging | ReAct | Step-by-step clarity |
| Batch processing | Static Planner | Pure efficiency |
| **Most real-world tasks** | **Hybrid** | **Handles uncertainty efficiently** |

---

### Design Principles:

1. **Keep mini-plans small (1-5 steps)**
   - Forces focused iterations
   - Enables frequent adaptation
   - Easier to debug

2. **Maximize parallelism within iterations**
   - Use empty `dependencies: []` for independent tasks
   - Let LLM identify parallelization opportunities
   - Leverage full CPU/network capacity

3. **Reflect thoroughly**
   - Synthesize ALL results from iteration
   - Identify patterns and problems
   - Inform next iteration's strategy

4. **Limit context window**
   - Keep last 2 iterations (not all history)
   - Use summaries for results
   - Balance context richness vs. token cost

---

## 🔧 Try It Yourself!

Experiment with these challenges:

### Challenge 1: Simple Parallel Math
```python
"Calculate 5 factorial and 10 factorial, then multiply the results together"
```
**Expected:** Iteration 1 runs both factorials in parallel, iteration 2 multiplies.

---

### Challenge 2: Mixed Research
```python
"Find the population of Tokyo and Paris, count letters in 'hello', then calculate the average population"
```
**Expected:** Iteration 1 parallelizes searches + string task, iteration 2 averages.

---

### Challenge 3: Complex Dependencies
```python
"Search for 'Flyte orchestration' and 'Python async', count words in each result, then write code to compare the word counts"
```
**Expected:** Iteration 1 parallelizes searches, iteration 2 parallelizes word counts (with dependencies on searches), iteration 3 compares.

---

### Challenge 4: Adaptive Discovery
```python
"Find the GDP of the top 3 economies, then calculate their total"
```
**Expected:** Agent identifies top 3, searches in parallel, then sums. Watch it adapt if search results are unclear!

---

Run your own experiments below:

In [ ]:
# Your experiment here!
!python -m workflows.flyte_react_planner --local --request "Your custom goal here" --max-iterations 8

---

## 🏆 Production Best Practices

### 1. **Tuning Max Iterations**
```python
# Simple tasks
--max-iterations 3

# Complex tasks
--max-iterations 10

# Research/exploration
--max-iterations 15
```

### 2. **Monitoring Execution**
- Check `total_iterations` in result
- Review `total_steps_executed` for efficiency
- Examine reflection quality for debugging

### 3. **Optimizing Prompts**
- Encourage parallelism: "Find A, B, C simultaneously"
- Be explicit about dependencies: "First X, then Y"
- Test with various phrasings

### 4. **Cost Optimization**
- Use gpt-4o for complex planning
- Use gpt-4o-mini for reflection
- Monitor token usage per iteration

### 5. **Error Handling**
- Set reasonable max_iterations
- Check `goal_achieved` flag
- Review logs for stuck patterns

---

## 🙏 Summary

Congratulations! You've mastered the **Hybrid ReAct + Planner pattern** - the ultimate multi-agent orchestration approach!

**What you learned:**
- 🔄 Iterative mini-planning for adaptability
- ⚡ Dependency-aware parallelism within iterations
- 🧠 Context accumulation and reflection
- 🎯 Balancing efficiency and flexibility
- 📊 Production-ready agent orchestration

**The power of Hybrid:**
> "Think adaptively at the macro level, execute efficiently at the micro level."

**You now have three patterns in your toolkit:**
1. 📋 **Static Planner** - Maximum speed for known tasks
2. 🔄 **ReAct** - Maximum flexibility for exploration
3. ⚡ **Hybrid** - Best of both worlds for production

---

## 🚀 What's Next?

Build your own agents:
- Add specialized agents (database, API, etc.)
- Create domain-specific tools
- Tune prompts for your use case
- Deploy to Flyte clusters for scale

**Remember:** The hybrid pattern is production-ready. Use it for real-world tasks that require both speed and adaptability!

---

Questions? Experiments? Go build something amazing! 🎉